In [1]:
from langchain_core.prompts.chat import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_openai import ChatOpenAI
import submit_code_tool
import query_tool
from langchain_community.chat_message_histories import RedisChatMessageHistory
from langchain.memory import ConversationBufferMemory


In [2]:
tools = [
   query_tool.reference_code_search,
   submit_code_tool.submit_modified_tf
  ]

message_history = RedisChatMessageHistory(
    url="redis://localhost:6379/0", ttl=600, session_id="my-session"
)

memory = ConversationBufferMemory(
    memory_key="chat_history", chat_memory=message_history, return_messages=True, human_prefix="human", ai_prefix="ai", input_key="input", output_key="output"
)

prompt = ChatPromptTemplate.from_messages([
  ("system", "You are an AWS solutions architect. Your job is to help customers build and deploy secure applications on AWS. Follow architecture best practices and find out approriate requirements before suggesting a solution. Once a tool call fails, stop trying. Once you have enough information, submit a proposal for a modified terraform file using the submit code tool."),
  ("human", "My current terraform file:\n{tf_file}"),
  MessagesPlaceholder("chat_history", optional=True),
  ("human", "{input}"),
  "{agent_scratchpad}"
])


llm = ChatOpenAI(model="gpt-4-1106-preview", temperature=.5)
# Construct the OpenAI Tools agent
agent = create_openai_tools_agent(llm, tools, prompt) # type: ignore # wrong type for query_tool import

# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, memory=memory, max_iterations=8, early_stopping_method="generate", return_intermediate_steps=True) # type: ignore # wrong type for query_tool import

In [3]:
agent_executor.invoke({
  "input": "I need a maanged postgres database without specific requirements. No high availability, but at least daily backups.",
  "tf_file": "```terraform resource \"aws_instance\" \"example\" { ami = \"${data.aws_ami.ubuntu.id}\" instance_type = \"t3.micro\" }```",
})



> Entering new AgentExecutor chain...

Invoking: `reference_code_search` with `{'query': 'aws_rds_instance postgres'}`


# Configure the AWS provider
provider "aws" {
  region = "eu-west-1"
}

# Create a DB instance
resource "aws_db_instance" "example" {
  engine              = "mysql"
  allocated_storage   = 10
  instance_class      = "db.t2.micro"
  name                = "example_database_stage"
  username            = "admin"
  password            = "${var.db_password}"
  skip_final_snapshot = true
}

Invoking: `submit_modified_tf` with `{'new_code': 'provider "aws" {\n  region = "eu-west-1"\n}\n\n# Create a PostgreSQL DB instance\nresource "aws_db_instance" "example_postgres" {\n  engine               = "postgres"\n  instance_class       = "db.t2.micro"\n  allocated_storage    = 20 # The minimum storage for PostgreSQL\n  name                 = "example_database_stage"\n  username             = "admin"\n  password             = "your_password" # Replace with a secure password or u

{'input': 'I need a maanged postgres database without specific requirements. No high availability, but at least daily backups.',
 'tf_file': '```terraform resource "aws_instance" "example" { ami = "${data.aws_ami.ubuntu.id}" instance_type = "t3.micro" }```',
 'chat_history': [HumanMessage(content='Please help me connec a database to my instance.'),
  AIMessage(content="To connect a database to your AWS instance, you would typically provision a database instance using a service such as Amazon RDS or create an EC2 instance and manually install and configure the database software. Then, you would configure your application to connect to the database using the database instance's connection details.\n\nHowever, before I can provide you with a modified Terraform file, I need to know a few more details:\n\n1. What type of database do you want to use (e.g., MySQL, PostgreSQL, etc.)?\n2. Do you prefer a managed database service like Amazon RDS, or do you want to install the database on an EC2 